In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
import nltk 
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize, sent_tokenize 
import os
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
stop_words = set(stopwords.words('english')) 
import numpy as np
from keras.models import Sequential
from keras.layers import Embedding
from keras.layers import Input
from keras.models import Model
from keras import optimizers
from keras.layers import concatenate,CuDNNLSTM
from keras import initializers
from keras import backend as K
import random

from keras.layers import Dense, LSTM,Dropout, Activation,Bidirectional,Reshape,Permute,Multiply,Flatten,Lambda,Layer,TimeDistributed,CuDNNLSTM

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


Using TensorFlow backend.


In [0]:
def SentenceToPOS(sentences):
    data = []
    for sentence in sentences: 
        one_sentence_pos = [] 
        
        wordsList = nltk.word_tokenize(sentence)
        
        wordsList = [w for w in wordsList if not w in stop_words]

        tagged = nltk.pos_tag(wordsList)
        
        for val in tagged:
            one_sentence_pos.append(val[1])
        data.append(one_sentence_pos)
    return data

def Fianl_X_and_Y(novelPOS, novelLabel, sentenceSize):
    i, X, Y = 0, [], []
    for novel in novelPOS:
        start, end = 0, sentenceSize
        while(end <= len(novel)):
            X.append(novel[start:end])
            Y.append(novelLabel[i])
            start = end
            end += sentenceSize 
        i += 1
    return X, Y

def Get_X_and_Y_In_POS_Form(fullPath, sentenceSize):
    filenameList = os.listdir(fullPath)
    filenameList.sort()
    
    novelPOS, novelLabel, fileNumber = [], [], 1
    for filename in filenameList:
        print(fileNumber,"/",len(filenameList)," Done")
        fileNumber += 1
        Path = fullPath + filename
        with open(Path, 'r') as f:
            data = f.read().replace('"\n"','').replace('\n',' ').replace('- ','')
            sentences = sent_tokenize(data)
            POS = SentenceToPOS(sentences)
            novelPOS.append(POS)
            novelLabel.append(filename[8])
    return Fianl_X_and_Y(novelPOS, novelLabel, sentenceSize)

In [4]:
print("Preprocessing Phase 1 : Fetching data")
print(".....................................")

X, Y = Get_X_and_Y_In_POS_Form('/content/gdrive/My Drive/dataset/',100)

print("Preprocessing Phase 1 : Finished\n\n")

Preprocessing Phase 1 : Fetching data
.....................................
1 / 33  Done
2 / 33  Done
3 / 33  Done
4 / 33  Done
5 / 33  Done
6 / 33  Done
7 / 33  Done
8 / 33  Done
9 / 33  Done
10 / 33  Done
11 / 33  Done
12 / 33  Done
13 / 33  Done
14 / 33  Done
15 / 33  Done
16 / 33  Done
17 / 33  Done
18 / 33  Done
19 / 33  Done
20 / 33  Done
21 / 33  Done
22 / 33  Done
23 / 33  Done
24 / 33  Done
25 / 33  Done
26 / 33  Done
27 / 33  Done
28 / 33  Done
29 / 33  Done
30 / 33  Done
31 / 33  Done
32 / 33  Done
33 / 33  Done
Preprocessing Phase 1 : Finished




In [5]:
maxval, minval = 0, 999999
for doc in X:
  for sentence in doc:
    minval = min(minval, len(sentence))
    maxval = max(maxval, len(sentence))
print(maxval, minval)

155 1


In [0]:
Labels = np.unique(Y, return_counts = True)[0]

In [0]:
LabelCount = {}
for label in Labels:
  LabelCount[label] = 0

In [0]:
MIN_VAL = min(np.unique(Y,return_counts=True)[1]) + 200

In [0]:
def dataBalancing(X, Y, LabelCount, MIN_VAL):
  X_data = []
  for i in range(len(X)):
      if(LabelCount[Y[i]] < MIN_VAL):
          X_data.append([X[i],Y[i]])
          LabelCount[Y[i]] += 1

  random.shuffle(X_data)

  new_X = []
  new_Y = []
  for m ,n in X_data:
    new_X.append(m)
    new_Y.append(n)

  return new_X, new_Y

In [0]:
X, Y = dataBalancing(X, Y, LabelCount, MIN_VAL)

In [0]:
def tag_to_index_dictionary(X):
    tag = set([])
    for doc in X:
        for sentence in doc:
            for word in sentence:
                tag.add(word)
    tag2index = {t: i + 1 for i, t in enumerate(list(tag))}
    tag2index['-PAD-'] = 0
    return tag2index

def convert_tag_to_sequence_numbers(X, tag2index):
    new_X=[]
    for doc in X:
        new_S = []
        for sentence in doc:
            new_W = []
            for word in sentence:
                new_W.append(tag2index[word])
            new_S.append(new_W)
        new_X.append(new_S)
    return new_X


def pad_zeros_to_sequence(X, max_length):
    new_X = []
    for doc in X:
        new_X.append(pad_sequences(doc, maxlen=max_length, padding='post'))
    return new_X

def Encode_Labels(Y):
    le = LabelEncoder()
    return le.fit_transform(Y)

In [0]:
tag2index = tag_to_index_dictionary(X)
DICTIONARY_LENGTH = len(tag2index)
MAX_LENGTH = 30 # why 30 only ?
SENTENCES_IN_NOVEL = 100

X = convert_tag_to_sequence_numbers(X, tag2index)
X = pad_zeros_to_sequence(X, MAX_LENGTH)
X = np.array(X)
TOTAL_ROWS = X.shape[0]

In [0]:
# My Code Above

In [14]:
A = [[[[1,2,3],[4,5,6],[7,8,9]],['A']],[[[12,2,3],[42,5,6],[72,8,9]],['B']],[[[11,2,3],[41,5,6],[71,8,9]],['C']]]
random.shuffle(A)
A

[[[[11, 2, 3], [41, 5, 6], [71, 8, 9]], ['C']],
 [[[12, 2, 3], [42, 5, 6], [72, 8, 9]], ['B']],
 [[[1, 2, 3], [4, 5, 6], [7, 8, 9]], ['A']]]

In [0]:
real_X = []
for i in range(SENTENCES_IN_NOVEL):
    real_X.append(list())
for i in range(SENTENCES_IN_NOVEL):
    for j in range(TOTAL_ROWS):
        real_X[i].append(X[j][i])

X_train = []
for i in range(SENTENCES_IN_NOVEL):
    X_train.append(np.array(real_X[i]))

Y = Encode_Labels(Y) 
from tensorflow.keras.utils import to_categorical
y_binary = to_categorical(Y)

In [16]:
np.array(y_binary).shape

(1803, 14)

In [0]:
from keras.engine.topology import Layer as LL
class Attention(LL):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3
        print(self.W_constraint)
        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))
        print(eij.shape,self.b.shape)

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim

In [26]:
from keras import  regularizers,constraints,initializers
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, SpatialDropout1D
from keras.layers import Dense,Input,LSTM,Bidirectional,Activation,Conv1D,GRU

model = Sequential() 

FIRST_LSTM = 256
SECOND_LSTM = 256
EMBEDD_VECTOR_LENGTH = 128
outputs=[]
inputs_=[]
EMB = Embedding(DICTIONARY_LENGTH, EMBEDD_VECTOR_LENGTH, input_length=30)

# BDR1 = SpatialDropout1D(0.2)
BDR2 = Bidirectional(GRU(FIRST_LSTM, return_sequences=True,dropout=0.1,recurrent_dropout=0.1))
BDR3 = Conv1D(FIRST_LSTM, kernel_size = 3, padding = "valid", kernel_initializer = "glorot_uniform")
BDR4 = GlobalAveragePooling1D()
BDR5 = GlobalMaxPooling1D()


# BDR = Bidirectional(CuDNNLSTM(FIRST_LSTM,return_sequences=True),merge_mode='concat')
TDB = TimeDistributed(Dense(DICTIONARY_LENGTH+3))
for i in range(SENTENCES_IN_NOVEL): # SENTENCES_IN_NOVEL = 100
    if(i % 10 == 0):
      print(i,"/",SENTENCES_IN_NOVEL)
    inputlayer = Input(shape=[MAX_LENGTH]) # MAX_LENGTH = 30 (30 words per sentence)
    inputs_.append(inputlayer)
    layer = EMB(inputlayer) # 50 is embedd vector size for each POS Tag
#     layer =  BDR1(layer) # output 100 because of bidirectonal concat
    layer =  BDR2(layer)
    layer =  BDR3(layer)
    avg_pool =  BDR4(layer)
    max_pool =  BDR5(layer)
    layer = concatenate([avg_pool, max_pool])
    outputs.append(layer)

merge_ = concatenate(outputs)
print(np.array(outputs).shape)
merge_ = Reshape((SENTENCES_IN_NOVEL, 2*FIRST_LSTM), input_shape=(SENTENCES_IN_NOVEL*(2*FIRST_LSTM),))(merge_)
merge_ = Bidirectional(CuDNNLSTM(SECOND_LSTM,return_sequences=True),merge_mode='concat')(merge_)

attention_mul = Attention(100)(merge_)#step_dimen =100
output = Dense(14, activation='softmax')(attention_mul)

model = Model(inputs=inputs_,outputs=output)
model.summary()

0 / 100
10 / 100
20 / 100
30 / 100
40 / 100
50 / 100
60 / 100
70 / 100
80 / 100
90 / 100
(100,)
None
(?, 100) (100,)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_372 (InputLayer)          (None, 30)           0                                            
__________________________________________________________________________________________________
input_373 (InputLayer)          (None, 30)           0                                            
__________________________________________________________________________________________________
input_374 (InputLayer)          (None, 30)           0                                            
__________________________________________________________________________________________________
input_375 (InputLayer)          (None, 30)           0                                     

In [27]:
 np.array([1,2,3,4,5,6]).reshape(2,3)

array([[1, 2, 3],
       [4, 5, 6]])

In [28]:
np.array(X_train).shape

(100, 1803, 30)

In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Nadam(lr=0.001),
              metrics=['accuracy'])
history = model.fit(X_train, y_binary, epochs=30, validation_split=0.1)

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 1622 samples, validate on 181 samples
Epoch 1/30
1622/1622 [==============================] - 667s 411ms/step - loss: 2.3538 - acc: 0.2004 - val_loss: 1.8278 - val_acc: 0.3039
Epoch 2/30
1622/1622 [==============================] - 471s 290ms/step - loss: 1.6760 - acc: 0.3588 - val_loss: 1.5446 - val_acc: 0.4309
Epoch 3/30
1622/1622 [==============================] - 467s 288ms/step - loss: 1.4282 - acc: 0.4593 - val_loss: 1.6867 - val_acc: 0.4033
Epoch 4/30
1622/1622 [==============================] - 459s 283ms/step - loss: 1.2442 - acc: 0.5351 - val_loss: 1.2469 - val_acc: 0.5304
Epoch 5/30
1622/1622 [==============================] - 451s 278ms/step - loss: 1.1314 - acc: 0.5734 - val_loss: 1.0913 - val_acc: 0.5967
Epoch 6/30
1622/1622 [==============================] - 452s 279ms/step - loss: 1.0215 - acc: 0.6097 - val_loss: 1.1883 - val_acc: 0.5304

In [0]:
import pandas as pd
import matplotlib.pyplot as plt
hist = pd.DataFrame(history.history)
plt.figure(figsize=(12,12))
plt.plot(hist["acc"])
plt.plot(hist["val_acc"])
plt.show()

In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Nadam(lr=0.001),
              metrics=['accuracy'])
history2 = model.fit(X_train,y_binary,batch_size=16, epochs=30,validation_split=0.1)

In [0]:
hist = pd.DataFrame(history.history)
plt.figure(figsize=(12,12))
plt.plot(hist["acc"],label = "Training Accuracy")
plt.plot(hist["val_acc"],label = "Validation Accuracy")
plt.title('Epoc Number Vs Accuracy') 
plt.xlabel('Epoc Number') 
# naming the y axis 
plt.ylabel('Accuracy')   
# show a legend on the plot 
plt.legend() 
plt.show()